# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | monsters.ipynb
 |-data\Orcus - Monsters.xlsx
 |-output
```

and will take data\Orcus - Monsters.xlsx and create a markdown file output\monsters.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Monsters.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

,Chapter,Name,Alignment,Size,Ancestry,Type,Tag,Role,Rank,Leader?,...,Cha,Equipment,Description,Source,Str mod,Con mod,Dex mod,Int mod,Wis mod,Cha mod
0,a,a,a,a,a,a,a,a,a,a,...,a,a,a,a,a,a,a,a,a,a
1,Animals,Chapter,,,,,,,,,...,,,"Animals are mundane beasts, like those found i...",,,,,,,
2,Animals,Raven of Doom,Unaligned,Small,Natural,Beast,,Skulker,,,...,10,,,DCC 54,-1,2,3,-2,1,0
3,Animals,Eagle,Unaligned,Small,Natural,Beast,,Striker,,,...,13,,,DCC 57,1,2,3,-3,2,1
4,Animals,Rattlesnake Swarm,Unaligned,Medium,Natural,Beast,Swarm,Striker,,,...,10,,,MD M1,1,2,5,-3,2,1


In [3]:
#cell 2, examine 5 random monsters

data.sample(5)

,Chapter,Name,Alignment,Size,Ancestry,Type,Tag,Role,Rank,Leader?,...,Cha,Equipment,Description,Source,Str mod,Con mod,Dex mod,Int mod,Wis mod,Cha mod
73,Humans,Daggermaster,Any,Medium,Natural,Humanoid,Human,Blocker,,,...,10,knife,,UM,3,2,4,1,1,1
15,Animals,Quipper Swarm,Unaligned,Medium,Natural,Beast,"Aquatic, Swarm",Striker,,,...,12,,,DCC 58,4,6,7,-1,3,4
138,Ostovites,Ostovite Swarm,Evil,Large,Cosmic,Monstrosity,Swarm,Spoiler,Boss,,...,8,,,PF2B3,1,2,3,-4,1,-1
111,Invaders,Skinner,Unaligned,Medium,Cosmic,Automaton,Construct,Archer,Mook,,...,12,,Skinners are jumbled collections of cybernetic...,UM,8,8,11,9,8,7
105,Infected,Infected Drone,Unaligned,Medium,Natural,Humanoid,,Wrecker,,,...,3,,The infected drone was a normal human but has ...,UM,3,1,-1,-3,0,-2


In [4]:
#cell 3
#we define a function which takes in a row of this power dataframe and returns an md string

def monster_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
    if row_dict.get('Name') == 'Chapter':
        md = '# {Chapter}  '.format(**row_dict)
        if row['Description']:
            md += "\n{Description}  ".format(**row_dict)
        return md

    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
        
    #start with name. md is the string we'll be outputing
    md = """\n### {Name}  """.format(**row_dict)
    
    #this is a complex chunk
    md += """\n**Level {Level}""".format(**row_dict)
    
    if row['Rank']:
        md += """ {Rank} {Role}**""".format(**row_dict)
    else: 
        md += """ {Role}**""".format(**row_dict)
    
    if row['Leader?']:
        md += " (Leader)".format(**row_dict)
    
    md += """ ({XP} XP)  """.format(**row_dict)
    
    md += """\n{Size} {Ancestry} {Type}""".format(**row_dict)

    if row['Tag']:
        md += " ● {Tag}".format(**row_dict)
    
    md += """ ({Alignment})  """.format(**row_dict)

    md += """\n**Senses:** Perception +{Perception}""".format(**row_dict)
    if row['Senses']:
        md += ", {Senses}""".format(**row_dict)
    
    if row['Skills']:
        md += """; **Skills:** {Skills}""".format(**row_dict)
    
    md += """   """.format(**row_dict) 
    
    if row['Language']:
        md += """\n**Languages:** {Language}  """.format(**row_dict)

    md += """\n**Str:** {Str} (+{Str mod}), **Con:** {Con} (+{Con mod}), **Dex:** {Dex} (+{Dex mod})\n**Int:** {Int} (+{Int mod}), **Wis:** {Wis} (+{Wis mod}), **Cha:** {Cha} (+{Cha mod})  """.format(**row_dict)
    
    if row['Equipment']:
        md += """\n**Equipment:** {Equipment}  """.format(**row_dict)
    
    md += """\n""".format(**row_dict)
    
    md += """\n**Initiative:** +{Initiative}""".format(**row_dict)
    
    md += """; **Speed:** {Spd}  """.format(**row_dict)
    
    if row['Action Points']:
        md += """\n**Action Points:** {Action Points}  """.format(**row_dict)
    
    md += """\n**AC:** {AC}; **Fort:** {Fort}, **Ref:** {Ref}, **Will:** {Will}""".format(**row_dict)
    
    if row['Saving Throws']:
        md += """; **Saving Throws:** +{Saving Throws}""".format(**row_dict)
    
    md += """   """.format(**row_dict) 
    
    md += """\n**HP:** {HP}""".format(**row_dict)

    if row['Staggered']:
        md += """, **Staggered:** {Staggered}""".format(**row_dict)
    
    if row['HP Extras']:
        md += """; {HP Extras}""".format(**row_dict)
    
    md += """   """.format(**row_dict) 
    
    if row['Immunity to']:
        md += """\n""".format(**row_dict)
    elif row['Resistance to']:
        md += """\n""".format(**row_dict) 
    elif row['Vulnerability to']:
        md += """\n""".format(**row_dict) 

    if row['Immunity to']:
        md += """**Immune:**&nbsp;{Immunity to}""".format(**row_dict)
    
    if row['Resistance to']:
        if row['Immunity to']:
            md += """, """.format(**row_dict)
            
    if row['Resistance to']:
        md += """**Resist:**&nbsp;{Resistance to}""".format(**row_dict)
 
    if row['Vulnerability to']:
        if row['Resistance to']:
            md += """, """.format(**row_dict)
        elif row['Immunity to']: 
            md += """, """.format(**row_dict)
    
    if row['Vulnerability to']:
        md += """**Vulnerable:**&nbsp;{Vulnerability to}""".format(**row_dict)
    
    md += """   """.format(**row_dict) 

    if row['Aura']:
        md += """\n{Aura}  """.format(**row_dict)
    
    md += """\n""".format(**row_dict) 
    
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 1 - Tags'):
        row_dict['Power 1 - Tags'] = '● **{Power 1 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 1 - Range'):
        row_dict['Power 1 - Range'] = '{Power 1 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 1 - Attack Bonus'):
        row_dict['Power 1 - Attack Bonus'] = '+{Power 1 - Attack Bonus}'.format(**row_dict)
        
    # Need to untangle so Rider displayed regardless of Defense existing
    
    if row_dict.get('Power 1 - Type') == 'Melee':
        row_dict['Power 1 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 1 - Type') == 'Basic Melee':
        row_dict['Power 1 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 1 - Type') == 'Ranged':
        row_dict['Power 1 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 1 - Type') == 'Basic Ranged':
        row_dict['Power 1 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 1 - Type') == 'Near':
        row_dict['Power 1 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 1 - Type') == 'Far':
        row_dict['Power 1 - Type'] = '⋇	'.format(**row_dict)
        
#     if row_dict.get('Power 1 - Type') == 'Melee':
#         row_dict['Power 1 - Type'] = '⚔️ '.format(**row_dict)
 
#     if row_dict.get('Power 1 - Type') == 'Basic Melee':
#         row_dict['Power 1 - Type'] = '<span style="height: 25px; width: 25px; background-color: #fff; border: 1.25px solid #000; border-radius: 50%; display:inline-block;">⚔️</span> '.format(**row_dict)

#     if row_dict.get('Power 1 - Type') == 'Ranged':
#         row_dict['Power 1 - Type'] = '🏹 '.format(**row_dict)
 
#     if row_dict.get('Power 1 - Type') == 'Basic Ranged':
#         row_dict['Power 1 - Type'] = '<span style="height: 25px; width: 25px; background-color: #fff; border: 1.25px solid #000; border-radius: 50%; display:inline-block;">🏹</span> '.format(**row_dict)

#     if row_dict.get('Power 1 - Type') == 'Near':
#         row_dict['Power 1 - Type'] = '💥 '.format(**row_dict)

#     if row_dict.get('Power 1 - Type') == 'Far':
#         row_dict['Power 1 - Type'] = '💣 '.format(**row_dict)
        
    if row['Power 1 - Name']:
        md += """\n{Power 1 - Type}**{Power 1 - Name}**""".format(**row_dict)

    if row_dict.get('Power 1 - Frequency'):
        row_dict['Power 1 - Frequency'] = ', {Power 1 - Frequency}'.format(**row_dict)   
        
    if row['Power 1 - Action']:        
        md += """ ({Power 1 - Action}{Power 1 - Frequency})""".format(**row_dict)
        
    if row['Power 1 - Name']:
        md += """ {Power 1 - Tags}  """.format(**row_dict)
        md += """\n{Power 1 - Range}{Power 1 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 1 - Defense']:
        if row['Power 1 - Rider']:
            md += """ vs {Power 1 - Defense} ({Power 1 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 1 - Defense}; """.format(**row_dict)
    elif row['Power 1 - Rider']:
        md += """{Power 1 - Rider}; """.format(**row_dict)
    
    if row['Power 1 - Name']:
        md += """{Power 1 - Effect}  """.format(**row_dict)
    
    if row['Power 1 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 1 - Secondary Attack}  """.format(**row_dict)

        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 2 - Tags'):
        row_dict['Power 2 - Tags'] = '● **{Power 2 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 2 - Range'):
        row_dict['Power 2 - Range'] = '{Power 2 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 2 - Attack Bonus'):
        row_dict['Power 2 - Attack Bonus'] = '+{Power 2 - Attack Bonus}'.format(**row_dict)
 
    if row_dict.get('Power 2 - Type') == 'Melee':
        row_dict['Power 2 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 2 - Type') == 'Basic Melee':
        row_dict['Power 2 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 2 - Type') == 'Ranged':
        row_dict['Power 2 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 2 - Type') == 'Basic Ranged':
        row_dict['Power 2 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 2 - Type') == 'Near':
        row_dict['Power 2 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 2 - Type') == 'Far':
        row_dict['Power 2 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 2 - Name']:
        md += """\n{Power 2 - Type}**{Power 2 - Name}**""".format(**row_dict)

    if row_dict.get('Power 2 - Frequency'):
        row_dict['Power 2 - Frequency'] = ', {Power 2 - Frequency}'.format(**row_dict)   
        
    if row['Power 2 - Action']:        
        md += """ ({Power 2 - Action}{Power 2 - Frequency})""".format(**row_dict)
        
    if row['Power 2 - Name']:
        md += """ {Power 2 - Tags}  """.format(**row_dict)
        md += """\n{Power 2 - Range}{Power 2 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 2 - Defense']:
        if row['Power 2 - Rider']:
            md += """ vs {Power 2 - Defense} ({Power 2 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 2 - Defense}; """.format(**row_dict)
    elif row['Power 2 - Rider']:
        md += """{Power 2 - Rider}; """.format(**row_dict)
    
    if row['Power 2 - Name']:
        md += """{Power 2 - Effect}  """.format(**row_dict)
    
    if row['Power 2 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 2 - Secondary Attack}  """.format(**row_dict)

        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 3 - Tags'):
        row_dict['Power 3 - Tags'] = '● **{Power 3 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 3 - Range'):
        row_dict['Power 3 - Range'] = '{Power 3 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 3 - Attack Bonus'):
        row_dict['Power 3 - Attack Bonus'] = '+{Power 3 - Attack Bonus}'.format(**row_dict)
        
    if row_dict.get('Power 3 - Type') == 'Melee':
        row_dict['Power 3 - Type'] = '†	'.format(**row_dict)
    
    if row_dict.get('Power 3 - Type') == 'Basic Melee':
        row_dict['Power 3 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 3 - Type') == 'Ranged':
        row_dict['Power 3 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 3 - Type') == 'Basic Ranged':
        row_dict['Power 3 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 3 - Type') == 'Near':
        row_dict['Power 3 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 3 - Type') == 'Far':
        row_dict['Power 3 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 3 - Name']:
        md += """\n{Power 3 - Type}**{Power 3 - Name}**""".format(**row_dict)

    if row_dict.get('Power 3 - Frequency'):
        row_dict['Power 3 - Frequency'] = ', {Power 3 - Frequency}'.format(**row_dict)   
        
    if row['Power 3 - Action']:        
        md += """ ({Power 3 - Action}{Power 3 - Frequency})""".format(**row_dict)
        
    if row['Power 3 - Name']:
        md += """ {Power 3 - Tags}  """.format(**row_dict)
        md += """\n{Power 3 - Range}{Power 3 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 3 - Defense']:
        if row['Power 3 - Rider']:
            md += """ vs {Power 3 - Defense} ({Power 3 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 3 - Defense}; """.format(**row_dict)
    elif row['Power 3 - Rider']:
        md += """{Power 3 - Rider}; """.format(**row_dict)
    
    if row['Power 3 - Name']:
        md += """{Power 3 - Effect}  """.format(**row_dict)
        
    if row['Power 3 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 3 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 4 - Tags'):
        row_dict['Power 4 - Tags'] = '● **{Power 4 - Tags}**'.format(**row_dict)

    if row_dict.get('Power 4 - Range'):
        row_dict['Power 4 - Range'] = '{Power 4 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 4 - Attack Bonus'):
        row_dict['Power 4 - Attack Bonus'] = '+{Power 4 - Attack Bonus}'.format(**row_dict)
        
    if row_dict.get('Power 4 - Type') == 'Melee':
        row_dict['Power 4 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 4 - Type') == 'Basic Melee':
        row_dict['Power 4 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 4 - Type') == 'Ranged':
        row_dict['Power 4 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 4 - Type') == 'Basic Ranged':
        row_dict['Power 4 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 4 - Type') == 'Near':
        row_dict['Power 4 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 4 - Type') == 'Far':
        row_dict['Power 4 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 4 - Name']:
        md += """\n{Power 4 - Type}**{Power 4 - Name}**""".format(**row_dict)

    if row_dict.get('Power 4 - Frequency'):
        row_dict['Power 4 - Frequency'] = ', {Power 4 - Frequency}'.format(**row_dict)   
        
    if row['Power 4 - Action']:        
        md += """ ({Power 4 - Action}{Power 4 - Frequency})""".format(**row_dict)
        
    if row['Power 4 - Name']:
        md += """ {Power 4 - Tags}  """.format(**row_dict)
        md += """\n{Power 4 - Range}{Power 4 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 4 - Defense']:
        if row['Power 4 - Rider']:
            md += """ vs {Power 4 - Defense} ({Power 4 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 4 - Defense}; """.format(**row_dict)
    elif row['Power 4 - Rider']:
        md += """{Power 4 - Rider}; """.format(**row_dict)
    
    if row['Power 4 - Name']:
        md += """{Power 4 - Effect}  """.format(**row_dict)
        
    if row['Power 4 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 4 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 5 - Tags'):
        row_dict['Power 5 - Tags'] = '● **{Power 5 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 5 - Range'):
        row_dict['Power 5 - Range'] = '{Power 5 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 5 - Attack Bonus'):
        row_dict['Power 5 - Attack Bonus'] = '+{Power 5 - Attack Bonus}'.format(**row_dict)
        
    if row_dict.get('Power 5 - Type') == 'Melee':
        row_dict['Power 5 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 5 - Type') == 'Basic Melee':
        row_dict['Power 5 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 5 - Type') == 'Ranged':
        row_dict['Power 5 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 5 - Type') == 'Basic Ranged':
        row_dict['Power 5 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 5 - Type') == 'Near':
        row_dict['Power 5 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 5 - Type') == 'Far':
        row_dict['Power 5 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 5 - Name']:
        md += """\n{Power 5 - Type}**{Power 5 - Name}**""".format(**row_dict)

    if row_dict.get('Power 5 - Frequency'):
        row_dict['Power 5 - Frequency'] = ', {Power 5 - Frequency}'.format(**row_dict)   
        
    if row['Power 5 - Action']:        
        md += """ ({Power 5 - Action}{Power 5 - Frequency})""".format(**row_dict)
        
    if row['Power 5 - Name']:
        md += """ {Power 5 - Tags}  """.format(**row_dict)
        md += """\n{Power 5 - Range}{Power 5 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 5 - Defense']:
        if row['Power 5 - Rider']:
            md += """ vs {Power 5 - Defense} ({Power 5 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 5 - Defense}; """.format(**row_dict)
    elif row['Power 5 - Rider']:
        md += """{Power 5 - Rider}; """.format(**row_dict)
    
    if row['Power 5 - Name']:
        md += """{Power 5 - Effect}  """.format(**row_dict)
    
    if row['Power 5 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 5 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 6 - Tags'):
        row_dict['Power 6 - Tags'] = '● **{Power 6 - Tags}**'.format(**row_dict)

    if row_dict.get('Power 6 - Range'):
        row_dict['Power 6 - Range'] = '{Power 6 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 6 - Attack Bonus'):
        row_dict['Power 6 - Attack Bonus'] = '+{Power 6 - Attack Bonus}'.format(**row_dict)
    
    if row_dict.get('Power 6 - Type') == 'Melee':
        row_dict['Power 6 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 6 - Type') == 'Basic Melee':
        row_dict['Power 6 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 6 - Type') == 'Ranged':
        row_dict['Power 6 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 6 - Type') == 'Basic Ranged':
        row_dict['Power 6 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 6 - Type') == 'Near':
        row_dict['Power 6 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 6 - Type') == 'Far':
        row_dict['Power 6 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 6 - Name']:
        md += """\n{Power 6 - Type}**{Power 6 - Name}**""".format(**row_dict)

    if row_dict.get('Power 6 - Frequency'):
        row_dict['Power 6 - Frequency'] = ', {Power 6 - Frequency}'.format(**row_dict)   
        
    if row['Power 6 - Action']:        
        md += """ ({Power 6 - Action}{Power 6 - Frequency})""".format(**row_dict)
        
    if row['Power 6 - Name']:
        md += """ {Power 6 - Tags}  """.format(**row_dict)
        md += """\n{Power 6 - Range}{Power 6 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 6 - Defense']:
        if row['Power 6 - Rider']:
            md += """ vs {Power 6 - Defense} ({Power 6 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 6 - Defense}; """.format(**row_dict)
    elif row['Power 6 - Rider']:
        md += """{Power 6 - Rider}; """.format(**row_dict)
    
    if row['Power 6 - Name']:
        md += """{Power 6 - Effect}  """.format(**row_dict)
        
    if row['Power 6 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 6 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 7 - Tags'):
        row_dict['Power 7 - Tags'] = '● **{Power 7 - Tags}**'.format(**row_dict)

    if row_dict.get('Power 7 - Range'):
        row_dict['Power 7 - Range'] = '{Power 7 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 7 - Attack Bonus'):
        row_dict['Power 7 - Attack Bonus'] = '+{Power 7 - Attack Bonus}'.format(**row_dict)
    
    if row_dict.get('Power 7 - Type') == 'Melee':
        row_dict['Power 7 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 7 - Type') == 'Basic Melee':
        row_dict['Power 7 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 7 - Type') == 'Ranged':
        row_dict['Power 7 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 7 - Type') == 'Basic Ranged':
        row_dict['Power 7 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 7 - Type') == 'Near':
        row_dict['Power 7 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 7 - Type') == 'Far':
        row_dict['Power 7 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 7 - Name']:
        md += """\n{Power 7 - Type}**{Power 7 - Name}**""".format(**row_dict)

    if row_dict.get('Power 7 - Frequency'):
        row_dict['Power 7 - Frequency'] = ', {Power 7 - Frequency}'.format(**row_dict)   
        
    if row['Power 7 - Action']:        
        md += """ ({Power 7 - Action}{Power 7 - Frequency})""".format(**row_dict)
        
    if row['Power 7 - Name']:
        md += """ {Power 7 - Tags}  """.format(**row_dict)
        md += """\n{Power 7 - Range}{Power 7 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 7 - Defense']:
        if row['Power 7 - Rider']:
            md += """ vs {Power 7 - Defense} ({Power 7 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 7 - Defense}; """.format(**row_dict)
    elif row['Power 7 - Rider']:
        md += """{Power 7 - Rider}; """.format(**row_dict)
    
    if row['Power 7 - Name']:
        md += """{Power 7 - Effect}  """.format(**row_dict)
    
    if row['Power 7 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 7 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 8 - Tags'):
        row_dict['Power 8 - Tags'] = '● **{Power 8 - Tags}**'.format(**row_dict)

    if row_dict.get('Power 8 - Range'):
        row_dict['Power 8 - Range'] = '{Power 8 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 8 - Attack Bonus'):
        row_dict['Power 8 - Attack Bonus'] = '+{Power 8 - Attack Bonus}'.format(**row_dict)

    if row_dict.get('Power 8 - Type') == 'Melee':
        row_dict['Power 8 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 8 - Type') == 'Basic Melee':
        row_dict['Power 8 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 8 - Type') == 'Ranged':
        row_dict['Power 8 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 8 - Type') == 'Basic Ranged':
        row_dict['Power 8 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 8 - Type') == 'Near':
        row_dict['Power 8 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 8 - Type') == 'Far':
        row_dict['Power 8 - Type'] = '⋇	'.format(**row_dict)
             
    if row['Power 8 - Name']:
        md += """\n{Power 8 - Type}**{Power 8 - Name}**""".format(**row_dict)

    if row_dict.get('Power 8 - Frequency'):
        row_dict['Power 8 - Frequency'] = ', {Power 8 - Frequency}'.format(**row_dict)   
        
    if row['Power 8 - Action']:        
        md += """ ({Power 8 - Action}{Power 8 - Frequency})""".format(**row_dict)
        
    if row['Power 8 - Name']:
        md += """ {Power 8 - Tags}  """.format(**row_dict)
        md += """\n{Power 8 - Range}{Power 8 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 8 - Defense']:
        if row['Power 8 - Rider']:
            md += """ vs {Power 8 - Defense} ({Power 8 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 8 - Defense}; """.format(**row_dict)
    elif row['Power 8 - Rider']:
        md += """{Power 8 - Rider}; """.format(**row_dict)
    
    if row['Power 8 - Name']:
        md += """{Power 8 - Effect}  """.format(**row_dict)
    
    if row['Power 8 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 8 - Secondary Attack}  """.format(**row_dict)
    
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 9 - Tags'):
        row_dict['Power 9 - Tags'] = '● **{Power 9 - Tags}**'.format(**row_dict)

    if row_dict.get('Power 9 - Range'):
        row_dict['Power 9 - Range'] = '{Power 9 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 9 - Attack Bonus'):
        row_dict['Power 9 - Attack Bonus'] = '+{Power 9 - Attack Bonus}'.format(**row_dict)

    if row_dict.get('Power 9 - Type') == 'Melee':
        row_dict['Power 9 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 9 - Type') == 'Basic Melee':
        row_dict['Power 9 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 9 - Type') == 'Ranged':
        row_dict['Power 9 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 9 - Type') == 'Basic Ranged':
        row_dict['Power 9 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 9 - Type') == 'Near':
        row_dict['Power 9 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 9 - Type') == 'Far':
        row_dict['Power 9 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 9 - Name']:
        md += """\n{Power 9 - Type}**{Power 9 - Name}**""".format(**row_dict)

    if row_dict.get('Power 9 - Frequency'):
        row_dict['Power 9 - Frequency'] = ', {Power 9 - Frequency}'.format(**row_dict)   
        
    if row['Power 9 - Action']:        
        md += """ ({Power 9 - Action}{Power 9 - Frequency})""".format(**row_dict)
        
    if row['Power 9 - Name']:
        md += """ {Power 9 - Tags}  """.format(**row_dict)
        md += """\n{Power 9 - Range}{Power 9 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 9 - Defense']:
        if row['Power 9 - Rider']:
            md += """ vs {Power 9 - Defense} ({Power 9 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 9 - Defense}; """.format(**row_dict)
    elif row['Power 9 - Rider']:
        md += """{Power 9 - Rider}; """.format(**row_dict)
    
    if row['Power 9 - Name']:
        md += """{Power 9 - Effect}  """.format(**row_dict)
    
    if row['Power 9 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 9 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 10 - Tags'):
        row_dict['Power 10 - Tags'] = '● **{Power 10 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 10 - Range'):
        row_dict['Power 10 - Range'] = '{Power 10 - Range}; '.format(**row_dict)
    
    if row_dict.get('Power 10 - Attack Bonus'):
        row_dict['Power 10 - Attack Bonus'] = '+{Power 10 - Attack Bonus}'.format(**row_dict)
        
    if row_dict.get('Power 10 - Type') == 'Melee':
        row_dict['Power 10 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 10 - Type') == 'Basic Melee':
        row_dict['Power 10 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 10 - Type') == 'Ranged':
        row_dict['Power 10 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 10 - Type') == 'Basic Ranged':
        row_dict['Power 10 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 10 - Type') == 'Near':
        row_dict['Power 10 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 10 - Type') == 'Far':
        row_dict['Power 10 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 10 - Name']:
        md += """\n{Power 10 - Type}**{Power 10 - Name}**""".format(**row_dict)

    if row_dict.get('Power 10 - Frequency'):
        row_dict['Power 10 - Frequency'] = ', {Power 10 - Frequency}'.format(**row_dict)   
        
    if row['Power 10 - Action']:        
        md += """ ({Power 10 - Action}{Power 10 - Frequency})""".format(**row_dict)
        
    if row['Power 10 - Name']:
        md += """ {Power 10 - Tags}  """.format(**row_dict)
        md += """\n{Power 10 - Range}{Power 10 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 10 - Defense']:
        if row['Power 10 - Rider']:
            md += """ vs {Power 10 - Defense} ({Power 10 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 10 - Defense}; """.format(**row_dict)
    elif row['Power 10 - Rider']:
        md += """{Power 10 - Rider}; """.format(**row_dict)
    
    if row['Power 10 - Name']:
        md += """{Power 10 - Effect}  """.format(**row_dict)
    
    if row['Power 10 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 10 - Secondary Attack}  """.format(**row_dict)
    
    if row['Description']:
        md += """\n\n{Description}  """.format(**row_dict) 
            
    #finally we remove the empty tags
    return md.replace('+-','-')

In [5]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = monster_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [6]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')


### Longshot Berserker  
**Level 19 Mook Striker** (600 XP)  
Medium Natural Humanoid ● Human (Any)  
**Senses:** Perception +14; **Skills:** Acrobatics +18, Athletics +15, Intimidate +15   
**Languages:** Common  
**Str:** 14 (+11), **Con:** 14 (+11), **Dex:** 20 (+14)
**Int:** 16 (+12), **Wis:** 14 (+11), **Cha:** 12 (+10)  
**Equipment:** crossbow  

**Initiative:** +15; **Speed:**   
**AC:** 33; **Fort:** 33, **Ref:** 31, **Will:** 30   
**HP:** 1; a missed attack never damages a mook      

‡	**Longsword** (standard, at-will) ● **Weapon**  
+24 vs AC; 17 damage.  
⤢	**Crossbow** (standard, at-will) ● **Weapon**  
Ranged 15/30; +24 vs AC; 17 damage.  
**Resilient** (counter, encounter)   
The longshot berserker is hit with an attack: The attack misses.  

# Ichor-Ghouls  
Hundreds of years ago, a secret organization in pursuit of power made the mistake of combining two powerful magical items: an *orb of chaos* and the mysterious *necrosis cube*. The result was the creation of the 

In [7]:
#cell 6
# now we have to save to disk

fname =  'Orcus Monsters.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))